## 将bytes文件转化为bmp图像

In [1]:
import numpy as np
from PIL import Image
import pefile
import math
import os

In [2]:
bytes_path = "/share/mal/kaggle/train/8va102hpJn5DVLe9i6Fq.bytes"
bmp_path = "/root/test.bmp"

In [3]:
def bytes2bmp(bytes_path, bmp_path):
    """
    针对kaggle的格式
    """
    
    blist = [] # byte list 
    with open(bytes_path) as f:
        for line in f:
            words = line.split(' ')
            words = words[1:] # ignore the address column
            for word in words:
                if word not in ["??", "??\n"]: # filter "??"
                    blist.append(int(word, 16)) # conver hex string to int
    width = int(math.sqrt(len(blist)))
    if width == 0: # 如果bytes文件都是??，那就算了
        return
    b_mat = np.asarray(blist, dtype=np.uint8)
    height = len(b_mat) // width
    b_mat = b_mat[0:width*height]
    b_mat = np.reshape(b_mat, (-1, width)) 
    img = Image.fromarray(b_mat)
    img.save(bmp_path)
    return img

In [12]:
b_mat = bytes2bmp(bytes_path, bmp_path)

In [ ]:
b_mat

## walk转换bytes文件到bmp文件
大概每秒2/3个

In [4]:
path = "/share/mal/tmp/train"

In [5]:
def walk_trans(path):
    for root, _, fnames in os.walk(path):
        for fname in fnames:
            if fname.endswith('.bytes'):
                bytes_path = os.path.join(root, fname)
                bmp_path = os.path.join(root, fname.split('.')[-2] + '.bmp')
                if not os.path.exists(bmp_path): # 判重
                    bytes2bmp(bytes_path, bmp_path)

In [6]:
walk_trans(path)

## 将bmp转化为png

In [ ]:
png_path = "/root/test.png"

In [ ]:
def bmp2png(bmp_path, png_path):
    bmp = Image.open(bmp_path)
    bmp_mat = np.asarray(bmp, dtype=np.uint8)
    png_mat = np.reshape(bmp_mat, (bmp_mat.shape[0], bmp_mat.shape[1], 1))
    png_mat = np.repeat(png_mat, 3, axis=2) # 在第2个维度重复3遍即可
    png = Image.fromarray(png_mat, 'RGB')
    png.save(png_path)
    return png

In [ ]:
png = bmp2png(bmp_path, png_path)

In [ ]:
png

## 将pe转化为bmp

In [ ]:
pe_path = "/share/mal/tmp/da884a0416425450f8362811d5bd453ad559ef41ad343f354a71e469be72e55a"
bmp_path = "/share/mal/tmp/da884a0416425450f8362811d5bd453ad559ef41ad343f354a71e469be72e55a.bmp"

In [ ]:
def pe2bmp(pe_path, bmp_path):
    pe = pefile.PE(pe_path)
    selected_sections = [sec for sec in pe.sections if str(sec.Name, encoding='utf-8').strip('\x00') in ['.text', '.data', '.rdata']]
    selected_bytes = []
    last_addr = selected_sections[0].VirtualAddress
    for sec in selected_sections:
        if last_addr < sec.VirtualAddress:
            selected_bytes.extend([0] * (sec.VirtualAddress - last_addr))
            last_addr = sec.VirtualAddress
        selected_bytes.extend(sec.get_data())
        last_addr += sec.SizeOfRawData
    width = int(math.sqrt(len(selected_bytes)))
    b_mat = np.asarray(selected_bytes, dtype=np.uint8)
    b_mat = b_mat[0:len(b_mat)//width*width]
    b_mat= np.reshape(b_mat, (-1, width))
    img = Image.fromarray(b_mat)
    img.save(bmp_path)
    return img

In [ ]:
bmp = pe2bmp(pe_path, bmp_path)

In [ ]:
bmp

## 将asm文件只保留opcode部分
针对kaggle格式/IDA pro格式